# Логистическая регрессия: Южный Парк

Импортирую всякие модули

In [2]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

from nltk import word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

Вытаскиваю и нормирую данные.

In [3]:
SO_PARK = pandas.read_csv('All-seasons.csv', sep=',')
SO_PARK.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


Под главными персонажами я поняла четвёрку главных.

In [4]:
stan = SO_PARK[SO_PARK['Character'] == 'Stan']
kyle = SO_PARK[SO_PARK['Character'] == 'Kyle']
eric = SO_PARK[SO_PARK['Character'] == 'Cartman']
kenny = SO_PARK[SO_PARK['Character'] == 'Kenny']

In [5]:
print('stan: ' + str(len(stan)))
print('kyle: ' + str(len(kyle)))
print('eric: ' + str(len(eric)))
print('kenny: ' + str(len(kenny)))

stan: 7680
kyle: 7099
eric: 9774
kenny: 881


Данные несбалансированы, потому что Кенни говорит слишком мало. При этом данных очень много, потому что это все сезоны. Поэтому я, пожалуй, просто выкину часть лишних данных.

In [9]:
less_stan = stan.sample(n=len(kenny))
less_kyle = kyle.sample(n=len(kenny))
less_eric = eric.sample(n=len(kenny))
balanced = pandas.concat([less_stan, less_kyle, less_eric, kenny])
balanced.groupby('Character').describe()

Episode        Line Season
Character                                  
Cartman   count      881         881    881
          unique      18         875     18
          top          4  Butters.\n      4
          freq        75           3     79
Kenny     count      881         881    881
          unique      18         754     18
          top          3   (Yeah!)\n      3
          freq       106          17    132
Kyle      count      881         881    881
          unique      18         843     18
          top          9     What?\n      2
          freq        79          13     97
Stan      count      881         881    881
          unique      18         853     18
          top         10     What?\n      2
          freq        75           8     95

Теперь датасет сбалансирован.

## Baseline

In [73]:
import random

rand_names = ['Cartman', 'Kenny', 'Kyle', 'Stan'] * 881
random.shuffle(rand_names)
print(rand_names[:100])

['Kenny', 'Kyle', 'Kenny', 'Stan', 'Kenny', 'Kenny', 'Kyle', 'Cartman', 'Kenny', 'Kenny', 'Stan', 'Kenny', 'Stan', 'Cartman', 'Stan', 'Kenny', 'Kenny', 'Cartman', 'Kenny', 'Kyle', 'Stan', 'Cartman', 'Cartman', 'Kenny', 'Kyle', 'Kyle', 'Kyle', 'Kenny', 'Stan', 'Cartman', 'Cartman', 'Kenny', 'Kenny', 'Stan', 'Kyle', 'Cartman', 'Stan', 'Cartman', 'Stan', 'Kyle', 'Stan', 'Cartman', 'Stan', 'Kenny', 'Cartman', 'Kyle', 'Kenny', 'Kyle', 'Stan', 'Stan', 'Stan', 'Kenny', 'Kenny', 'Stan', 'Kyle', 'Kyle', 'Kyle', 'Kyle', 'Stan', 'Cartman', 'Kenny', 'Kyle', 'Cartman', 'Kenny', 'Cartman', 'Kenny', 'Stan', 'Kyle', 'Kenny', 'Kyle', 'Cartman', 'Cartman', 'Kenny', 'Stan', 'Kyle', 'Kenny', 'Kyle', 'Stan', 'Kenny', 'Cartman', 'Kyle', 'Kyle', 'Kenny', 'Kenny', 'Kyle', 'Kyle', 'Stan', 'Cartman', 'Kyle', 'Kenny', 'Stan', 'Kenny', 'Kyle', 'Kenny', 'Kenny', 'Cartman', 'Stan', 'Stan', 'Cartman', 'Kenny']


In [74]:
print(classification_report(balanced['Character'], rand_names))

             precision    recall  f1-score   support

    Cartman       0.27      0.27      0.27       881
      Kenny       0.27      0.27      0.27       881
       Kyle       0.25      0.25      0.25       881
       Stan       0.25      0.25      0.25       881

avg / total       0.26      0.26      0.26      3524



Если распределять героев с рандомно с одинаковой вероятностью, точность в райное 0.25, что полностью соответствует теории вероятностей.

## Preparing data

Дальше я попробую лемматизировать и не лемматизировать. Лемматизировать – потому что это кажется разумным решением, в конце концов, о человеке должны больше говорять лексемы, которые он употребляет, чем словоформы. Не лемматизировать, потому что в прошлом дз в задаче классификации писем внезапно лучше показали семя не лемматизированные словари.

In [39]:
# не лемматизируя
bow_non_lem = CountVectorizer()
bow_non_lem.fit_transform(balanced['Line'])
bowed_non_lem = bow_non_lem.transform(balanced['Line'])

In [41]:
# функция для лемматизации. коммент: я пыталась сделать это как полагается, и сначала сделать POS-tagging,
# но лемматизатор ворднета не идеален. он, например, вообще не умеет лемматизировать местоимения...

from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

def lemmatize(word):
    noun = lmtzr.lemmatize(word)
    verb = lmtzr.lemmatize(word, 'v')
    if verb != word:
        return verb
    return noun

def tokenize_lemmatize(text):
    text = word_tokenize(text.lower())
    return [lemmatize(word) for word in text if word not in '!?,.":;']

bow_lem = CountVectorizer(tokenizer=tokenize_lemmatize)
bow_lem.fit_transform(balanced['Line'])
bowed_lem = bow_lem.transform(balanced['Line'])

In [ ]:
Смотрю на частотные словари.

In [75]:
bow_lem.vocabulary_

{'waa-ooo': 3208,
 'weave': 3250,
 'manager': 1828,
 'fbi': 1078,
 'adverse': 119,
 'imaginary': 1547,
 'essay': 1003,
 'perfect': 2248,
 'argghh..': 222,
 'biblical': 355,
 'aclu': 99,
 'rap': 2443,
 'macroed': 1803,
 'dumbass': 916,
 'siren': 2693,
 'spirituality': 2788,
 'true': 3084,
 'morning': 1954,
 'next-gen': 2031,
 'u-uh': 3104,
 'thanksgiving': 2974,
 'lyric': 1790,
 'jeez': 1603,
 'earth': 932,
 'say': 2582,
 'gross': 1304,
 'these': 2986,
 'field': 1095,
 'tuh-': 3091,
 'dollar': 876,
 'story': 2843,
 'revenge': 2510,
 'yuh': 3422,
 'flashback': 1123,
 'jonas': 1626,
 'revamp': 2509,
 'funny': 1197,
 'rehab': 2474,
 'peel': 2240,
 'forget': 1155,
 'still': 2830,
 '-and': 31,
 'bullshit': 453,
 'visit': 3192,
 'strong': 2854,
 'tough': 3049,
 'planetarium': 2292,
 'them': 2980,
 'tap': 2928,
 'mcdickinass': 1855,
 'cousin': 711,
 'ms.': 1974,
 'action': 102,
 'smuggle': 2730,
 'eighty': 964,
 'workin': 3355,
 'o-kay': 2098,
 'shirt': 2656,
 'towards': 3054,
 'don': 878,
 'b

## Training

In [78]:
X_train, X_test, y_train, y_test = train_test_split(bowed_non_lem, 
                                                    balanced['Character'], 
                                                    test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2819, 3799) (705, 3799) (2819,) (705,)


In [ ]:
## Testing

In [ ]:
## Graphs